<a href="https://colab.research.google.com/github/SMSinclair/DS-Unit-1-Sprint-4-Statistical-Tests-and-Experiments/blob/master/Stephen_Sinclair_LS_DS_141_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
### YOUR CODE STARTS HERE
import pandas as pd
import numpy as np

In [43]:
# GOAL 1: Load and clean the data

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [0]:
!curl https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.names

In [0]:
feature_names = ['Class Name',
          'handicapped-infants',
   'water-project-cost-sharing',
   'adoption-of-the-budget-resolution',
   'physician-fee-freeze',
   'el-salvador-aid',
   'religious-groups-in-schools',
   'anti-satellite-test-ban',
   'aid-to-nicaraguan-contras',
   'mx-missile',
   'immigration',
   'synfuels-corporation-cutback',
   'education-spending',
   'superfund-right-to-sue',
   'crime',
   'duty-free-exports',
   'export-administration-act-south-africa'
                ]

In [45]:
df.columns = feature_names
df.head()

,Class Name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [32]:
df.loc[2][1]

'?'

In [46]:
df = df.replace({'?':np.nan, 'n':0, 'y':1})
df.head()

,Class Name,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [92]:
df["Class Name"].value_counts()

democrat      267
republican    168
Name: Class Name, dtype: int64

In [0]:
# GOAL 2: Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01

In [47]:
# Make a pivot table to check the relative levels of support for the 16 bills.

pd.pivot_table(data=df, index='Class Name')

,adoption-of-the-budget-resolution,aid-to-nicaraguan-contras,anti-satellite-test-ban,crime,duty-free-exports,education-spending,el-salvador-aid,export-administration-act-south-africa,handicapped-infants,immigration,mx-missile,physician-fee-freeze,religious-groups-in-schools,superfund-right-to-sue,synfuels-corporation-cutback,water-project-cost-sharing
Class Name,,,,,,,,,,,,,,,,
democrat,0.888462,0.828897,0.772201,0.350195,0.637450,0.144578,0.215686,0.935135,0.604651,0.471483,0.758065,0.054054,0.476744,0.289683,0.505882,0.502092
republican,0.134146,0.152866,0.240741,0.981366,0.089744,0.870968,0.951515,0.657534,0.187879,0.557576,0.115152,0.987879,0.897590,0.860759,0.132075,0.506757


In [54]:
# Democrats seemed to support the adoption of the budget resolution at a much
# higher rate than Republicans.

# Check the vote counts

df.groupby('Class Name')['adoption-of-the-budget-resolution'].value_counts()

Class Name  adoption-of-the-budget-resolution
democrat    1.0                                  231
            0.0                                   29
republican  0.0                                  142
            1.0                                   22
Name: adoption-of-the-budget-resolution, dtype: int64

In [0]:
from scipy.stats import ttest_ind

In [57]:
dem_budget = df[df['Class Name']=='democrat']['adoption-of-the-budget-resolution']
dem_budget.shape

(267,)

In [59]:
rep_budget = df[df['Class Name']=='republican']['adoption-of-the-budget-resolution']
rep_budget.shape

(168,)

In [61]:
ttest_ind(dem_budget, rep_budget, nan_policy='omit')

Ttest_indResult(statistic=23.21277691701378, pvalue=2.0703402795404463e-77)

In [0]:
statistic, pvalue = ttest_ind(dem_budget, rep_budget, nan_policy='omit')

In [68]:
# The p-value is well below the 0.01 threshold, so we can confidently reject the
# null hypothesis that Republicans and Democrats had the same average support 
# for the bill.

pvalue <= 0.01

True

In [0]:
# GOAL 3: Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01


In [69]:
dem_freeze = df[df["Class Name"]=='democrat']['physician-fee-freeze']
rep_freeze = df[df['Class Name']=='republican']['physician-fee-freeze']
dem_freeze.shape

(267,)

In [70]:
rep_freeze.shape

(168,)

In [73]:
ttest_ind(dem_freeze, rep_freeze, nan_policy='omit')

Ttest_indResult(statistic=-49.36708157301406, pvalue=1.994262314074344e-177)

In [74]:
# The p-value is well below 0.01, so we can reject the null hypothesis in this 
# case as well.

statistic, pvalue = ttest_ind(dem_freeze, rep_freeze, nan_policy='omit')
pvalue <= 0.01

True

In [75]:
# GOAL 4: Using hypothesis testing, find an issue where the difference between 
# republicans and democrats has p > 0.1 (i.e. there may not be much of a 
# difference)

dem_water = df[df["Class Name"]=='democrat']['water-project-cost-sharing']
rep_water = df[df['Class Name']=='republican']['water-project-cost-sharing']
dem_freeze.shape

(267,)

In [76]:
rep_water.shape

(168,)

In [80]:
statistic, pvalue = ttest_ind(dem_water, rep_water, nan_policy='omit')
pvalue

0.9291556823993485

In [81]:
# The p-value is way over 0.1, so there's no evidence of an underlying 
#difference in support between Democrats and Republicans

pvalue > 0.1

True

In [0]:
# Stretch goals:
# 1.) Refactor

# Define a function that takes a feature from the dataframe and a significance 
# level and returns a tuple with a boolean value (True if the pvalue < 
# significance level) and the pvalue.

def support_ttest(feature, level):
  dem = df[df["Class Name"]=='democrat'][feature]
  rep = df[df['Class Name']=='republican'][feature]
  statistic, pvalue = ttest_ind(dem, rep, nan_policy='omit')
  return pvalue < level, pvalue

In [87]:
support_ttest('water-project-cost-sharing', .05)

(False, 0.9291556823993485)

In [88]:
support_ttest('physician-fee-freeze', .01)

(True, 1.994262314074344e-177)

In [91]:
support_ttest('adoption-of-the-budget-resolution', .01)

(True, 2.0703402795404463e-77)